<a href=\
 target=\
><img src=\
 alt=\
/></a>

## Overview

This notebook demonstrates how to train a U-Net model with FiLM layers for segmentation. It begins with environment setup, verifies the data pipeline, initializes experiment settings, and finally runs the training loop with monitoring and checkpointing.

## Environment Setup

This section mounts Google Drive, clones the project repository, installs required packages, and links the dataset so training can run smoothly.

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
import os

COLAB_PROJECT_ROOT = '/content/Thesis_Project'

if not os.path.exists(COLAB_PROJECT_ROOT):

  !git clone https://github.com/nahubn1/Hybrid-Robot-Navigation-System {COLAB_PROJECT_ROOT}

os.chdir(COLAB_PROJECT_ROOT) # Change directory into the project

!git pull # Ensure it's the latest version

In [ ]:
!pip install -r environment/requirements.txt

In [ ]:
import sys

# Add src to path

sys.path.append(os.path.join(os.getcwd(), 'src'))



# Define Drive paths

DRIVE_DATA_PATH = '/content/drive/MyDrive/Thesis_DNN_Planner/data'

DRIVE_MODELS_PATH = '/content/drive/MyDrive/Thesis_DNN_Planner/models'

DRIVE_RESULTS_PATH = '/content/drive/MyDrive/Thesis_DNN_Planner/results'

LOCAL_DATA_PATH = '/content/dataset'



# Copy dataset to the local SSD for faster loading

if not os.path.exists(LOCAL_DATA_PATH):

    !rsync -ah --info=progress2 {DRIVE_DATA_PATH}/ {LOCAL_DATA_PATH}/

    !ls {LOCAL_DATA_PATH} | head

DATA_PATH = LOCAL_DATA_PATH



# Link Drive storage to local cloned directories

if not os.path.islink('data'):

    !ln -s {DATA_PATH} data

if not os.path.islink('models'):

    !ln -s {DRIVE_MODELS_PATH} models

if not os.path.islink('results'):

    !ln -s {DRIVE_RESULTS_PATH} results



print(\
)

## Phase 3: Experiment Setup and Initialization

Here we load configuration files, create dataset splits, and initialize the model, optimizer, and learning rate scheduler.

In [ ]:
import yaml

from pathlib import Path



CONFIG_PATH = 'configs/dnn/unet_film_v1_baseline.yaml'

with open(CONFIG_PATH, 'r') as f:

    cfg = yaml.safe_load(f)

print(f'Configuration loaded from {CONFIG_PATH}.')

In [ ]:
import random

import numpy as np

import torch



seed = cfg.get('seed', 0)

random.seed(seed)

np.random.seed(seed)

torch.manual_seed(seed)

if torch.cuda.is_available():

    torch.cuda.manual_seed_all(seed)

print(f'Random seed set to {seed}.')

In [ ]:
from sklearn.model_selection import train_test_split

from torch.utils.data import DataLoader

from dnn_guidance.data_loader import PathfindingDataset, _pair_files

from pathlib import Path



DATA_ROOT = Path(DATA_PATH)


In [ ]:
from dnn_guidance.model import UNetFiLM

from dnn_guidance.config import UNetConfig



model_cfg = UNetConfig.from_yaml('configs/dnn/unet_film.yaml')

model = UNetFiLM(model_cfg)


In [ ]:
from sklearn.model_selection import train_test_split

from torch.utils.data import DataLoader

from dnn_guidance.data_loader import PathfindingDataset, _pair_files

from pathlib import Path



DATA_ROOT = Path(DATA_PATH)


## Phase 4: Training Loop with Monitoring and Checkpointing

The training loop iterates over epochs while logging metrics to TensorBoard and saving checkpoints for recovery and evaluation.

In [ ]:
from datetime import datetime

from pathlib import Path

from torch.utils.tensorboard import SummaryWriter

from torch.cuda.amp import GradScaler

from dnn_guidance.trainer import train_one_epoch, validate_one_epoch



# Directories for logging and checkpoints

timestamp = datetime.now().strftime(\
)
